Persistence storing the messages in RAM where it stores the messages upto RAM holds the program

In [2]:
# Short Term Memory using InSaveMemory

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph,END,START,add_messages
from typing import TypedDict,Annotated,List
from pydantic import BaseModel
from langchain_core.messages import BaseMessage,HumanMessage
import time

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
# 2. Initialize Gemini 2.5 Pro
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.7,
)

c:\Users\Admin\Desktop\MyDesktop\Langchain\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm.invoke("HI how are you?")

AIMessage(content="Hello! I'm doing great, thank you for asking.\n\nI'm ready to help with any questions or tasks you have. How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--a760985b-d1ca-41d4-80fd-05e30ec39b34-0', usage_metadata={'input_tokens': 6, 'output_tokens': 33, 'total_tokens': 1122, 'input_token_details': {'cache_read': 0}})

In [4]:
class State(TypedDict):
    messages:Annotated[List[BaseMessage],add_messages]

In [5]:
def llm_node(state:State):
    messages=state['messages']
    response=llm.invoke(messages)
    return{
        'messages':response
    }

In [6]:
memory=InMemorySaver()

In [ ]:
graph=StateGraph(state_schema=State)

graph.add_node('llm_node',llm_node)

graph.add_edge(START,'llm_node')
graph.add_edge('llm_node',END)

chatbot=graph.compile(checkpointer=memory)



In [8]:
config={"configurable": {"thread_id": "1"}}

In [9]:
while(True):
    prompt=input()
    print(f"User:{prompt}")
    if prompt.strip().lower() in ['exit','bye','break']:
        break
    initial_state={
        'messages':[HumanMessage(content=prompt)]
    }
    response=chatbot.invoke(initial_state,config=config)
    print(f"AI:{response['messages'][-1].content}")

User:Hi my name is baba
AI:Hello Baba! It's nice to meet you.

How can I help you today?
User:can you tell me my name
AI:Yes, you told me your name is Baba.
User:add 100 +20
AI:100 + 20 = 120
User:and then add 40 to the result
AI:Of course.

120 + 40 = 160.
User:exist


In [12]:
# Fault Tolerarence

# Just put the None in input and pass the same thread id to continue where it stopped

In [3]:
class State(TypedDict):
    input: str
    step1: str
    step2: str

In [4]:
# 2. Define steps
def step_1(state: State) -> State:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: State) -> State:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: State) -> State:
    print("✅ Step 3 executed")
    return {"done": True}

In [5]:
# 3. Build the graph
builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
